<a href="https://colab.research.google.com/github/Jrodos12/Organizacion-de-Datos/blob/main/Analisis%20exploratorio%20de%20datos/16sparks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10 categorías que tienen la menor cantidad de contenido anónimo publicado. (⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=0bf89d91306459e89f839fd86de1a778d540f22df8ca75bd7e45ab73d4fef1c2
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_contents = sqlContext.read.csv("drive/MyDrive/contents.csv", header=True, inferSchema=True)
df_categorias = sqlContext.read.csv("drive/MyDrive/categorylinks.csv", header=True, inferSchema=True)
rdd_contents = df_contents.select("title", "id", "revisor_username", "revisor_id").rdd
rdd_categorias = df_categorias.select("cl_from", "cl_to").rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd_contents.first()

Row(title='Wikipedia:Artículos solicitados', id='5', revisor_username='Metrónomo', revisor_id='1708233')

In [ ]:
rdd_categorias.first()

Row(cl_from='5', cl_to='Wikipedia:Mantenimiento')

In [ ]:
rdd_anonimos = rdd_contents.filter(lambda x: x.revisor_username == None).map(lambda x: (x.id, x))

In [ ]:
rdd_categorias = rdd_categorias.filter(lambda x: x.cl_from != None).map(lambda x: (x.cl_from, x))

In [ ]:
rdd_join = rdd_anonimos.join(rdd_categorias).cache()

In [ ]:
lista = rdd_join.groupByKey().mapValues(len).collect()

In [ ]:
lista.sort(key=lambda a: a[1])

In [ ]:
for x in range(10):
  print(lista[x])



('775', 1)
('8972', 1)
('10061', 1)
('11676', 1)
('11967', 1)
('20608', 1)
('24632', 1)
('25650', 1)
('30643', 1)
('41513', 1)
